In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

def calculate_displacement(ref_frame, target_frame, enhance_contrast=True):
    # Convert frames to grayscale
    ref_gray = cv2.cvtColor(ref_frame, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target_frame, cv2.COLOR_BGR2GRAY)

    if enhance_contrast:
        ref_gray = cv2.equalizeHist(ref_gray)
        target_gray = cv2.equalizeHist(target_gray)

    # Calculate the absolute difference
    displacement = cv2.absdiff(ref_gray, target_gray)

    # Optionally apply thresholding or other processing here

    return displacement

def generate_heatmap(displacement_array, sensitivity=2):
    # Adjust normalization based on sensitivity
    norm = Normalize(vmin=np.min(displacement_array)/sensitivity, vmax=np.max(displacement_array)*sensitivity)
    colormap = plt.cm.magma

    heatmap = colormap(norm(displacement_array), bytes=False)
    heatmap_8bit = (heatmap * 255).astype(np.uint8)
    heatmap_bgr = cv2.cvtColor(heatmap_8bit, cv2.COLOR_RGBA2BGR)

    return heatmap_bgr

def upscale_frame(frame, scale_percent):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)

def process_video(input_video_path, output_video_path, upscale_percent=200):
    cap = cv2.VideoCapture(input_video_path)
    ret, ref_frame = cap.read()  # Assuming the first frame as the reference frame

    if not ret:
        cap.release()
        raise ValueError("Failed to read the first frame from the video.")

    # Upscale the reference frame
    ref_frame = upscale_frame(ref_frame, upscale_percent)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (ref_frame.shape[1], ref_frame.shape[0]))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Upscale each frame
        frame = upscale_frame(frame, upscale_percent)

        # Calculate displacement on the current frame
        displacement = calculate_displacement(ref_frame, frame)
        heatmap = generate_heatmap(displacement)
        out.write(heatmap)

    cap.release()
    out.release()

# Example usage
process_video('matchstick_test_2.mov', 'test_output.mov')